In [1]:
!pip install ultralytics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 KB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 25.3 MB/s eta 0:00:00


In [2]:
## importing required libraries
import os
import shutil
import random

!pip install tqdm --upgrade
from tqdm.notebook import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:

train_path_img = "./yolo_data/images/train/"
train_path_label = "./yolo_data/labels/train/"
val_path_img = "./yolo_data/images/val/"
val_path_label = "./yolo_data/labels/val/"
test_path = "./yolo_data/test"

In [5]:

'''
Split the dataset into train and test and creates the train.txt and test.tx with
the respective path of the images in each folder
'''

def train_test_split(path,neg_path=None, split = 0.2):
    print("------ PROCESS STARTED -------")


    files = list(set([name[:-4] for name in os.listdir(path)])) ## removing duplicate names i.e. counting only number of images
    

    print (f"--- This folder has a total number of {len(files)} images---")
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    ## creating required directories

    os.makedirs(train_path_img, exist_ok = True)
    os.makedirs(train_path_label, exist_ok = True)
    os.makedirs(val_path_img, exist_ok = True)
    os.makedirs(val_path_label, exist_ok = True)

    
    ### ----------- copying images to train folder
    for filex in tqdm(files[:train_size]):
      if filex == 'classes':
          continue
      shutil.copy2(path + filex + '.jpg',f"{train_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{train_path_label}/" + filex + '.txt')
        
    

    print(f"------ Training data created with 80% split {len(files[:train_size])} images -------")
    
    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)])) ## removing duplicate names i.e. counting only number of images
        for filex in tqdm(neg_images):
            shutil.copy2(neg_path+filex+ ".jpg", f"{train_path_img}/" + filex + '.jpg')
            
        print(f"------ Total  {len(neg_images)} negative images added to the training data -------")
    
        print(f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images -------")
    


    ### copytin images to validation folder
    for filex in tqdm(files[train_size:]):
      if filex == 'classes':
          continue
      # print("running")
      shutil.copy2(path + filex + '.jpg', f"{val_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{val_path_label}/" + filex + '.txt')

    print(f"------ Testing data created with a total of {len(files[train_size:])} images ----------")
    
    print("------ TASK COMPLETED -------")

## spliting the data into train-test and creating train.txt and test.txt files
# train_test_split('/content/drive/MyDrive/custom_notebooks/yolo_data/')

### for label_tag
train_test_split('/content/drive/MyDrive/face_detection/data/') ### without negative images
# train_test_split('./data/','./negative_images/') ### if you want to feed negative images

------ PROCESS STARTED -------
--- This folder has a total number of 31 images---


  0%|          | 0/25 [00:00<?, ?it/s]

------ Training data created with 80% split 25 images -------


  0%|          | 0/6 [00:00<?, ?it/s]

------ Testing data created with a total of 6 images ----------
------ TASK COMPLETED -------


In [6]:
# I will write the contents of the cell to a file
%%writefile /content/drive/MyDrive/face_detection/dataset.yaml 

# Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
#path:  # dataset root dir
train: /content/yolo_data/images/train # train images (relative to 'path')
val: /content/yolo_data/images/val # val images (relative to 'path')
test:  # test images (optional)

# number of classes
nc: 0

names: [ 'face']  

Overwriting /content/drive/MyDrive/face_detection/dataset.yaml


In [7]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.48 🚀 Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 25.8/78.2 GB disk)


In [8]:
!yolo task=detect mode=train model=yolov8s.pt data=/content/drive/MyDrive/face_detection/dataset.yaml epochs=25 imgsz=640 batch=8 project=/content/drive/MyDrive/face_detection/training_results name=face

100% 21.5M/21.5M [00:01<00:00, 16.4MB/s]
Ultralytics YOLOv8.0.48 🚀 Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/face_detection/dataset.yaml, epochs=25, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/face_detection/training_results, name=face, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, min_memory=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=Fals

In [9]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/face_detection/training_results/face/weights/best.pt conf=0.55 source=/content/drive/MyDrive/face_detection/test_images save=True

Ultralytics YOLOv8.0.48 🚀 Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs

image 1/11 /content/drive/MyDrive/face_detection/test_images/202211_jpg.rf.f29efb65c0106d4164c28156644abd2c.jpg: 640x640 (no detections), 17.2ms
image 2/11 /content/drive/MyDrive/face_detection/test_images/202218-1-_jpg.rf.6bbe3ca43209b98987ae91298c529993.jpg: 640x640 (no detections), 17.2ms
image 3/11 /content/drive/MyDrive/face_detection/test_images/202241_jpg.rf.e7e7db934d2511b0774d9f1ebdbfe8ca.jpg: 640x640 1 face, 17.2ms
image 4/11 /content/drive/MyDrive/face_detection/test_images/202247_jpg.rf.c0734f3dc4f782ea6908e5a3f52e72ba.jpg: 640x640 (no detections), 17.3ms
image 5/11 /content/drive/MyDrive/face_detection/test_images/20225--2-_jpg.rf.05ffec048f985b8132e52872c3bc5e7d.jpg: 640x640 (no detections), 17.2ms
image 6/11 /content/drive/MyDrive/face_detection/test_images/202252_jpg.rf.da8de86620a5b7c0f2d0e588b4dcfafb.j

In [10]:
!cp -r /content/runs/detect/predict /content/drive/MyDrive/face_detection/output

In [11]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/face_detection/training_results/face/weights/best.pt conf=0.55 source=/content/drive/MyDrive/face_detection/vid save=True

Ultralytics YOLOv8.0.48 🚀 Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs

video 1/1 (1/456) /content/drive/MyDrive/face_detection/vid/IMG_6406.MOV: 640x384 1 face, 25.5ms
video 1/1 (2/456) /content/drive/MyDrive/face_detection/vid/IMG_6406.MOV: 640x384 1 face, 15.7ms
video 1/1 (3/456) /content/drive/MyDrive/face_detection/vid/IMG_6406.MOV: 640x384 1 face, 15.7ms
video 1/1 (4/456) /content/drive/MyDrive/face_detection/vid/IMG_6406.MOV: 640x384 (no detections), 15.5ms
video 1/1 (5/456) /content/drive/MyDrive/face_detection/vid/IMG_6406.MOV: 640x384 1 face, 15.5ms
video 1/1 (6/456) /content/drive/MyDrive/face_detection/vid/IMG_6406.MOV: 640x384 (no detections), 18.7ms
video 1/1 (7/456) /content/drive/MyDrive/face_detection/vid/IMG_6406.MOV: 640x384 (no detections), 15.8ms
video 1/1 (8/456) /content/drive/MyDrive/face_detection/vid/IMG_6406.MOV: 640x384 (no detections), 15.6ms
video 1/1 (9/456) /

In [12]:
!cp -r /content/runs/detect/predict2 /content/drive/MyDrive/face_detection/output